In [1]:
filepath = '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC'

In [2]:
import os

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(filepath):
    for file in f:
        if '._' not in file and '.xml' in file and file != 'bookId.xml':
            files.append(os.path.join(r, file))

print(len(files))
print(files[0:5])

3998
['/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/35392.xml', '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/97486.xml', '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/35386.xml', '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/92850.xml', '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/36842.xml']


In [37]:
import xmltodict, json

# from anytree import AnyNode, RenderTree
bookTitles = {}
chapterTitles = {}
    
def singleChapter(INchapter, INparent):
    _title = INchapter['title']
    
    _from = ""
    if '@from' in INchapter:
        _from = INchapter['@from']
    
    _to = ""
    if '@to' in INchapter:
        _to = INchapter['@to']
        
    _level = ""
    if '@level' in INchapter:
        _level = INchapter['@level']
        
    _n = ""
    if '@n' in INchapter:
        _n = INchapter['@n']

    # 'parent' = [sub]
    if INparent not in chapterTitles:
        chapterTitles[INparent] = [_title]
    elif _title not in chapterTitles[INparent]:
        chapterTitles[INparent].append(_title)
        
    # [sub] = [parent]
#     if _title not in chapterTitles:
#         chapterTitles[_title] = [INparent]
#     elif INparent not in chapterTitles[_title]:
#         chapterTitles[_title].append(INparent)
    
def section(INarray, INtitle):
    if type(INarray) == dict:
        singleChapter(INarray, INtitle)
        
    elif type(INarray) == list:    
        for ch in INarray:
            section(ch, INtitle)
            if 'section' in ch:
                section(ch['section'], ch['title'])
                
    else:
        print(INarray)
        print("ERROR:", type(INarray))

# file: [35393].xml
def recursiveSection(INdata, fid):
    book = INdata['title']
    
    bookTitles[fid] = [book]
    if 'section' in INdata:
        section(INdata['section'], book)

def parseFile(file):
    with open(file) as fp:
        xmlStr = fp.read()
        convertedDict = xmltodict.parse(xmlStr);
        jsonStr = json.dumps(convertedDict, ensure_ascii=False)
        jsonObj = json.loads(jsonStr)

#         print(jsonObj)
        book = jsonObj['book']
    
        rstart = file.rfind('/')
        rend = file.rfind('.')
        # file: [35393].xml
        book['id'] = file[rstart+1:rend]
#         print(jsonObj)
        recursiveSection(book, file[rstart+1:rend])
        
print(files[0])
parseFile(files[0])
print(bookTitles)
print(chapterTitles)

/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/35392.xml
{'35392': ['開原縣志']}
{'開原縣志': ['鑛物（闕）', '卷首', '地理一二卷', '人物三四卷', '政治五六七卷', '人事八九卷', '物産十卷', '藝文十一十二卷'], '鑛物（闕）': ['園蔬植物（闕）'], '卷首': ['攝影', '重修開原縣志序', '章邑侯原序', '明邑侯原序', '胡廣文原序', '續修開原縣志序', '編輯縣志姓名錄', '開原縣志凡例', '全志目錄', '圖'], '地理一二卷': ['地理：附建置沿革表', '疆域區村：附區村表', '山脈：附境内諸山考', '河流', '城池公廨：附城池圖', '壇廟', '交通：附開豐汽車公司表 長途電話價目表', '商埠：附民六民十七比較表 市政公所一覽表', '古蹟：附名勝'], '人物三四卷': ['名宦', '鄉宦：附生存文武職', '選舉', '職官：列表', '科第：列表 附學校畢業一覽表', '忠節', '孝義', '儒林', '文苑', '鄉型', '藝術', '列女', '流寓'], '政治五六七卷': ['自治', '行政司法', '警察', '保甲', '戶口', '教育', '財賦', '邊防'], '財賦': ['國家稅', '地方稅'], '人事八九卷': ['種族', '氏族', '禮俗：附神道', '宗教', '兵事', '實業', '慈善', '外僑事略', '災異', '雜錄'], '物産十卷': ['動物', '植物'], '動物': ['有脊椎動物', '節足動物', '軟體動物', '蠕形動物', '海綿動物'], '植物': ['原生動物', '農作植物', '觀賞植物', '林果植物', '藥用植物', '食用隱花植物'], '藝文十一十二卷': ['文', '詩', '詞', '歌']}


In [30]:
# root = AnyNode(id="root")
for file in files:
    parseFile(file)
bookTitles
# parseFile(files[0])

{'35392': ['開原縣志'],
 '97486': ['建寧縣志'],
 '35386': ['新民縣志'],
 '92850': ['襄城縣志'],
 '36842': ['弋陽縣志'],
 '17494': ['襄陽府志'],
 '97860': ['增修萬縣志'],
 '97478': ['天長縣志纂輯志稿'],
 '17053': ['寧海縣志'],
 '97687': ['雩都縣志'],
 '36499': ['廣寧縣志'],
 '94371': ['安東縣志'],
 '93813': ['瑞金縣志'],
 '35783': ['黃陂縣志'],
 '278': ['聞喜縣志'],
 '97450': ['揚州府志'],
 '93637': ['吳川縣志'],
 '94372': ['宜春縣志'],
 '97343': ['續修臨沂縣志'],
 '92892': ['武城縣志'],
 '97082': ['繁峙縣志'],
 '35583': ['懷仁縣新志'],
 '97726': ['新化縣志'],
 '17291': ['昌樂縣志'],
 '35225': ['蒙陰縣志'],
 '97125': ['青田縣志'],
 '93152': ['靈州志迹'],
 '97524': ['慈谿縣志'],
 '36504': ['長春縣志'],
 '92933': ['鄖縣志'],
 '92538': ['高唐州志'],
 '35409': ['凌雲縣志'],
 '838': ['連山縣志'],
 '36821': ['續修臺灣府噶瑪蘭廳志'],
 '37043': ['綏陽縣志'],
 '16957': ['貴州通志'],
 '94313': ['鎮江府志'],
 '653': ['常熟縣志'],
 '36841': ['玉山縣志'],
 '71908': ['無錫志'],
 '92417': ['吉安府龍泉縣重修縣志'],
 '94370': ['臨安縣志'],
 '93087': ['重刊襄陽郡志'],
 '93091': ['直隸通州志'],
 '97724': ['綏寧縣志'],
 '38470': ['寧河縣志'],
 '17378': ['南安府志'],
 '92588': ['廣昌縣志'],
 '97709': ['茶陵縣志'],
 '934

In [31]:
len(bookTitles)

3998

In [5]:
# # Remove bookname
# booknamecnt = 0
# for sec in chapterTitles:
# #     print(type(sec))
#     if sec == None:
#         continue
#     titles = chapterTitles[sec]
    
#     for tit in titles:
#         if tit == None:
#             continue
# #         print(sec)
#         if sec.startswith(tit):
#             if tit not in bookTitles:
#                 # remove the same section names in parent and sub section both
#                 if sec == tit:
#                     chapterTitles[sec].remove(tit)
#                     print(sec, tit)
#             else:
# #                 print(tit)
#                 chapterTitles[sec].remove(tit)

#             booknamecnt += 1
        
#         # remove parent section is book
# #         if tit in bookTitles:
# #             chapterTitles[sec].remove(tit)
# #             booknamecnt += 1
# #             print(sec, tit)
            
# print(booknamecnt)
# # print(len(chapterTitles))

# Write chapters to file

In [6]:
allChapters = []

for key in chapterTitles:
    if key not in allChapters:
        allChapters.append(key)
    for c in chapterTitles[key]:
        if c not in allChapters:
            allChapters.append(c)

print(len(allChapters))

77425


In [7]:
import pandas as pd

resDf = pd.DataFrame.from_dict(allChapters)
# print(resDf)
resDf.sort_values(by=[0],inplace=True)
resDf.head()

,0
33834,(二)河流
58589,□制
53167,□陵墓志
51844,○丘墓
51875,○事紀


In [8]:
outputPath = '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志'
write_file_to = '{0}/{1}'.format(outputPath, 'allchapters_20190819_v2.xlsx')
resDf.to_excel(write_file_to, index=True)

# test

In [9]:
dupbooks = {}
for key in bookTitles:
#     print(bookTitles[key])
    if len(bookTitles[key]) > 1:
#         print(key, bookTitles[key])
        dupbooks[key] = bookTitles[key]
print(len(dupbooks))

861


In [10]:
import pandas as pd

resDf = pd.DataFrame.from_dict(dupbooks, orient='index')
# print(resDf)
resDf.sort_index(inplace=True)
resDf.head()

print(len(resDf))

861


In [11]:
outputPath = '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志'
write_file_to = '{0}/{1}'.format(outputPath, 'gazetteers_20190811_v2.xlsx')
resDf.to_excel(write_file_to, index=True)

In [12]:
# import pandas as pd

# resDf = pd.DataFrame(chapterTitles, columns=['section'])
# resDf.sort_values(by=['section'], inplace=True)
# resDf.head()

# print(len(resDf))

In [13]:
# outputPath = '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志'
# write_file_to = '{0}/{1}'.format(outputPath, 'gazetteers_20190811_v1.xlsx')
# resDf.to_excel(write_file_to, index=False)

In [14]:
count = 0
for key in chapterTitles:
    print(key, chapterTitles[key])
    count += 1
    if count > 5:
        break

園蔬植物（闕） ['鑛物（闕）']
攝影 ['卷首', '畫']
重修開原縣志序 ['卷首']
章邑侯原序 ['卷首']
明邑侯原序 ['卷首']
胡廣文原序 ['卷首']


In [15]:
# Remove bookname
booknamecnt = 0
for sec in chapterTitles:
#     print(type(sec))
    if sec == None:
        continue
    titles = chapterTitles[sec]
    
    for tit in titles:
        if tit == None:
            continue
#         print(sec)
        if sec.startswith(tit):
            if tit not in bookTitles:
                # remove the same section names in parent and sub section both
                if sec == tit:
                    chapterTitles[sec].remove(tit)
                    print(sec, tit)
            else:
#                 print(tit)
                chapterTitles[sec].remove(tit)

            booknamecnt += 1
        
        # remove parent section is book
#         if tit in bookTitles:
#             chapterTitles[sec].remove(tit)
#             booknamecnt += 1
#             print(sec, tit)
            
print(booknamecnt)
# print(len(chapterTitles))

詩 詩
序 序
藝文 藝文
凡例 凡例
武備志 武備志
學校 學校
兵制 兵制
水利 水利
兵衛 兵衛
官師 官師
進士 進士
目錄 目錄
秩官 秩官
舊序 舊序
教育志 教育志
科舉 科舉
郵舍 郵舍
祭 祭
祀事志 祀事志
舊跋 舊跋
重修邵武縣志序 重修邵武縣志序
沛縣志目次 沛縣志目次
城隍 城隍
都隅 都隅
市巷 市巷
正志校堪記 正志校堪記
南漳縣志集鈔卷之二十二 南漳縣志集鈔卷之二十二
南漳縣志集鈔卷之二十三 南漳縣志集鈔卷之二十三
南漳縣志集鈔卷之二十四 南漳縣志集鈔卷之二十四
襄陽沿革略 襄陽沿革略
息縣志卷之五 息縣志卷之五
366


In [16]:
bookTitles['成安縣志']

['94377', '97288']

In [17]:
# remove parent section is book
for sec in chapterTitles:
#     print(type(sec))
    if sec == None:
        continue
    titles = chapterTitles[sec]
    
    for tit in titles:
        if tit == None:
            continue

        if tit in bookTitles:
            # remove the parent section name which is the same as bookname
            chapterTitles[sec].remove(tit)
            
print(len(chapterTitles))

39049


In [19]:
# 僊居水卷之一
type(chapterTitles)

dict

In [ ]:
import re

for key in chapterTitles:
    titles = chapterTitles[key]
    for t in titles:
        x = re.search(".*卷[首之上下一二三四五六七八九十]*", t)
        if x != None:
            print(t, x)
            chapterTitles[key].remove(t)

In [ ]:
chapterTitles